In [ ]:
import os
import sys
from pathlib import Path
import jpype

if not jpype.isJVMStarted():
    jpype.startJVM(jpype.getDefaultJVMPath(), "-ea")

jpype.addClassPath("/home/michael/Documents/pcg/CMPUT652FinalProject/Mario-AI-Framework/src") # TODO: figure how to use relative path here
main = jpype.JClass('PythonController')

# Run a sample to test jpype is set up correctly
main.reset(False)
result = main.step([True, False, False, False, True])

# Print out screen observation
for i in range(result.observation.length):
            for j in range(result.observation[i].length):
                print(result.observation[i][j], end=" ")
            print("")



In [ ]:
from mario_env import MarioEnv
env = MarioEnv()

print(env.observation_space.shape)
print(env.reset().shape)

In [ ]:
from stable_baselines3.common.env_checker import check_env
from mario_env import MarioEnv
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, DummyVecEnv
from gym.wrappers import ResizeObservation

env = MarioEnv(render=False)

# It will check your custom environment and output additional warnings if needed
check_env(env)

In [ ]:
# Sample 100 random actions in the environment

from mario_env import MarioEnv
import numpy as np

env = MarioEnv()

for i in range(1):
    done = False
    env.reset()
    while not done:
        obs, reward, done, info = env.step(6 + np.random.randint(4)) # sample randomly from right-inputs Non


# --- Training an Agent ---

In [1]:
from stable_baselines3 import DQN, A2C, PPO
from mario_env import MarioEnv
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, DummyVecEnv
from gym.wrappers import ResizeObservation
from stable_baselines3.common.callbacks import CheckpointCallback

env = MarioEnv(render=False)

# Perform some wrapping on the environment
print(env.observation_space.shape)
env = ResizeObservation(env, 84) # Resize observations from 16x16 to 84x84
print(env.observation_space.shape)
env = DummyVecEnv([lambda: env]) # turn into vectorized environment
print(env.observation_space.shape)
env = VecFrameStack(env, n_stack=4) # stack vectorized environment by 4 frames
print(env.observation_space.shape)
env = VecTransposeImage(env, skip=False) # needed for images to properly be processed by CnnPolicy
print(env.observation_space.shape)


checkpoint_callback = CheckpointCallback(
  save_freq=25000,
  save_path="./logs/",
  name_prefix="rl_model",
  save_replay_buffer=True,
  save_vecnormalize=True,
)

model = PPO('CnnPolicy', env, verbose=1)
model.learn(total_timesteps=100000, callback=checkpoint_callback)
model.save("ppo_agent_100000")


/home/michael/Documents/pcg/CMPUT652FinalProject/Mario-AI-Framework/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(16, 16, 1)
(84, 84, 1)
(84, 84, 1)
(84, 84, 4)
(4, 84, 84)
Using cpu device
-----------------------------
| time/              |      |
|    fps             | 225  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 2048 |
-----------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 83        |
|    iterations           | 2         |
|    time_elapsed         | 49        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.4265379 |
|    clip_fraction        | 0.697     |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.09     |
|    explained_variance   | 6.14e-05  |
|    learning_rate        | 0.0003    |
|    loss                 | 119       |
|    n_updates            | 10        |
|    policy_gradient_loss | 0.0874    |
|    value_loss           | 474       |
---------------------------------

# --- Testing an Agent ---

In [1]:
from stable_baselines3 import DQN, A2C, PPO
from mario_env import MarioEnv
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, DummyVecEnv
from gym.wrappers import ResizeObservation

# Generate environment and wrap it
env = MarioEnv(render=True)
env = ResizeObservation(env, 84)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)
env = VecTransposeImage(env, skip=False)

model = PPO.load("logs/rl_model_100000_steps.zip") # Load agent

episodes = 1

for i in range(episodes):
    obs = env.reset()
    done = False
    while not done:
        action = model.predict(obs, deterministic=False)
        obs, reward, done, info = env.step(action)
        # print(reward) 
# env.close()

/home/michael/Documents/pcg/CMPUT652FinalProject/Mario-AI-Framework/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


java.lang.NullPointerException: java.lang.NullPointerException

In [ ]:
# SAMPLE OF CUSTOM CNN POLICY (NOT USED ANYMORE)

# import gym
# import torch as th
# import torch.nn as nn

# from stable_baselines3 import DQN
# from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# from mario_env import MarioEnv
# from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage
# from  stable_baselines3.common.vec_env import DummyVecEnv


# class CustomCNN(BaseFeaturesExtractor):
#     """
#     :param observation_space: (gym.Space)
#     :param features_dim: (int) Number of features extracted.
#         This corresponds to the number of unit for the last layer.
#     """

#     def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
#         super(CustomCNN, self).__init__(observation_space, features_dim)
#         # We assume CxHxW images (channels first)
#         # Re-ordering will be done by pre-preprocessing or wrapper
#         n_input_channels = observation_space.shape[0]
#         self.cnn = nn.Sequential(
#             nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
#             nn.ReLU(),
#             nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
#             nn.ReLU(),
#             nn.Flatten(),
#         )

#         # Compute shape by doing one forward pass
#         with th.no_grad():
#             n_flatten = self.cnn(
#                 th.as_tensor(observation_space.sample()[None]).float()
#             ).shape[1]

#         self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

#     def forward(self, observations: th.Tensor) -> th.Tensor:
#         return self.linear(self.cnn(observations))


# env = MarioEnv(render=False)
# print(env.observation_space.shape)
# env = DummyVecEnv([lambda: env])
# print(env.observation_space.shape)
# env = VecFrameStack(env, n_stack=4)
# print(env.observation_space.shape)
# env = VecTransposeImage(env, skip=True)
# print(env.observation_space.shape)

# policy_kwargs = dict(
#     features_extractor_class=CustomCNN,
#     features_extractor_kwargs=dict(features_dim=128),
# )
# model = DQN("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=1)
# model.learn(50000)
# model.save('cnn_50k_timesteps')